In [140]:
# Import Dependencies
import sqlalchemy
from sqlalchemy import create_engine

In [141]:
# Connect to DB
login = 'postgres:postgres'
db_url = 'postgresql://' + login + '@localhost:5432/Bitcoin_Nasdaq'
engine = create_engine(db_url)
connection = engine.connect()

In [142]:
# Reflect DB into a new model
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

# Show tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

['nasdaqs', 'bitcoin']

In [143]:
# Save references to each table
Bitcoin = Base.classes.bitcoin
Nasdaqs = Base.classes.nasdaqs

In [144]:
# Create link to SQL DB
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
session = Session(engine)
inspector = inspect(engine)

In [145]:
# Inspector
columns = inspector.get_columns('bitcoin')
for column in columns:
        print(column["name"], column["type"])

date DATE
opening_price NUMERIC
high NUMERIC
low NUMERIC
closing_price NUMERIC


In [146]:
# Inspector
columns = inspector.get_columns('nasdaqs')
for column in columns:
        print(column["name"], column["type"])

date DATE
opening_price NUMERIC
high NUMERIC
low NUMERIC
closing_price NUMERIC
adj_closing NUMERIC
volume BIGINT


In [147]:
# Session
for row in session.query(Bitcoin).limit(10).all():
        print(row.date)

2013-09-30
2013-10-01
2013-10-02
2013-10-03
2013-10-04
2013-10-05
2013-10-06
2013-10-07
2013-10-08
2013-10-09


In [148]:
# Session
for row in session.query(Nasdaqs).limit(10).all():
        print(row.date)

1971-02-05
1971-02-08
1971-02-09
1971-02-10
1971-02-11
1971-02-12
1971-02-16
1971-02-17
1971-02-18
1971-02-19


In [149]:
# Query to retrieve the last 12 months of precipitation data and plot the results
last_date_year= session.query(Bitcoin.date).order_by(Bitcoin.date.desc()).first()
print(last_date_year)

(datetime.date(2021, 2, 4),)


In [150]:
# Calculate the date 1 year ago from the last data point in the database
import datetime as dt
year_ago = dt.date(2021, 2, 4) - dt.timedelta(days=365)
print(year_ago)

2020-02-05


In [151]:
# Perform a query to retrieve the prices
bit_prices = session.query(Bitcoin.date,Bitcoin.closing_price).filter(Bitcoin.date <= dt.date(2021, 2, 4),Bitcoin.date >= dt.date(2020, 2, 5)).all()
print(prices)

           closing_price
date                    
2020-02-05   9625.456637
2020-02-06   9681.377411
2020-02-07   9795.344066
2020-02-08   9927.778256
2020-02-09    10132.7065
...                  ...
2021-01-31   33087.36986
2021-02-01   33613.32076
2021-02-02   35632.90195
2021-02-03   37397.42636
2021-02-04   37256.25211

[366 rows x 1 columns]


In [152]:
# Save results in DF
import pandas as pd
bit_prices_df= pd.DataFrame(bit_prices)
bit_prices_df.set_index("date", inplace=True)
bit_prices_df.head()

,closing_price
date,
2020-02-05,9625.456637
2020-02-06,9681.377411
2020-02-07,9795.344066
2020-02-08,9927.778256
2020-02-09,10132.7065


In [153]:
# Use same dates for Nasdaq
ndaq_prices = session.query(Nasdaqs.date,Nasdaqs.closing_price).filter(Nasdaqs.date <= dt.date(2021, 2, 4),Nasdaqs.date >= dt.date(2020, 2, 5)).all()
print(ndaq_prices)

[(datetime.date(2020, 2, 5), Decimal('9508.679688')), (datetime.date(2020, 2, 6), Decimal('9572.150391')), (datetime.date(2020, 2, 7), Decimal('9520.509766')), (datetime.date(2020, 2, 10), Decimal('9628.389648')), (datetime.date(2020, 2, 11), Decimal('9638.94043')), (datetime.date(2020, 2, 12), Decimal('9725.959961')), (datetime.date(2020, 2, 13), Decimal('9711.969727')), (datetime.date(2020, 2, 14), Decimal('9731.179688')), (datetime.date(2020, 2, 18), Decimal('9732.740234')), (datetime.date(2020, 2, 19), Decimal('9817.179688')), (datetime.date(2020, 2, 20), Decimal('9750.969727')), (datetime.date(2020, 2, 21), Decimal('9576.589844')), (datetime.date(2020, 2, 24), Decimal('9221.280273')), (datetime.date(2020, 2, 25), Decimal('8965.610352')), (datetime.date(2020, 2, 26), Decimal('8980.780273')), (datetime.date(2020, 2, 27), Decimal('8566.480469')), (datetime.date(2020, 2, 28), Decimal('8567.370117')), (datetime.date(2020, 3, 2), Decimal('8952.169922')), (datetime.date(2020, 3, 3), Deci

In [154]:
# Save results in DF
ndaq_prices_df= pd.DataFrame(ndaq_prices)
ndaq_prices_df.set_index("date", inplace=True)
ndaq_prices_df.head()

,closing_price
date,
2020-02-05,9508.679688
2020-02-06,9572.150391
2020-02-07,9520.509766
2020-02-10,9628.389648
2020-02-11,9638.94043


In [155]:
# Calculate summary statistics for NDAQ
ndaq_prices_df.describe()

,closing_price
count,249
unique,249
top,10550.49023
freq,1


In [156]:
# Calculate the summary statistics for Bitcoin
bit_prices_df.describe()

,closing_price
count,366
unique,366
top,7875.750875
freq,1


In [157]:
# Bitcoin- Mean, Median, Max, Min between 2020-2021
mean_bit = bit_prices_df.mean()
mean_bit

closing_price    13649.986839
dtype: float64

In [158]:
median_bit = bit_prices_df.median()
median_bit

closing_price    10481.28415
dtype: float64

In [159]:
max_bit = bit_prices_df.max()
max_bit

closing_price    40519.4486
dtype: float64

In [160]:
min_bit = bit_prices_df.min()
min_bit

closing_price    4944.702336
dtype: float64

In [161]:
# Nasdaq- Mean, Median, Max, Min between 2020-2021
mean_ndaq = ndaq_prices_df.mean()
mean_ndaq

closing_price    10517.609269
dtype: float64

In [162]:
median_ndaq = ndaq_prices_df.median()
median_ndaq

closing_price    10745.26953
dtype: float64

In [163]:
max_ndaq = ndaq_prices_df.max()
max_ndaq

closing_price    13635.99023
dtype: float64

In [164]:
min_ndaq = ndaq_prices_df.min()
min_ndaq

closing_price    6860.669922
dtype: float64

In [165]:
# Import Dependencies
%matplotlib inline
import matplotlib.pyplot as plt